In [1]:
import pandas as pd
import pybedtools
from matplotlib_venn import venn3
%matplotlib inline

Let's pull out significantly differentially expressed genes.

In [2]:
deseq2_dir = "/oasis/tscc/scratch/biom200/featurecounts/"
deseq2_result = pd.read_csv(deseq2_dir+"differential_expression.csv", index_col=0)
deseq2_result.head()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
ENSG00000223972.4,1.548277,-2.371333,2.573424,-0.921470,0.356805,NaN
ENSG00000227232.4,188.017621,0.104251,0.246626,0.422707,0.672509,0.823875
ENSG00000243485.2,0.263215,1.442268,4.982647,0.289458,0.772231,NaN
ENSG00000237613.2,4.917684,0.479224,1.333292,0.359430,0.719274,NaN
ENSG00000238009.2,155.704964,-0.305639,0.250297,-1.221106,0.222046,0.420962


GeneID isn't really helpful, let's add the gene name onto the dataframe. 

In [4]:
peak_dir = "/oasis/tscc/scratch/biom200/fto_clip/"

gene_names = pd.read_table(peak_dir+"gencode.v19.annotation.genenames.txt", index_col=0)
gene_names.head()

,sym
id,
ENSG00000223972.4,DDX11L1
ENSG00000227232.4,WASH7P
ENSG00000243485.2,MIR1302-11
ENSG00000237613.2,FAM138A
ENSG00000268020.2,OR4G4P


In [5]:
deseq2_result = deseq2_result.join(gene_names)
deseq2_result.head()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,sym
ENSG00000223972.4,1.548277,-2.371333,2.573424,-0.921470,0.356805,NaN,DDX11L1
ENSG00000227232.4,188.017621,0.104251,0.246626,0.422707,0.672509,0.823875,WASH7P
ENSG00000243485.2,0.263215,1.442268,4.982647,0.289458,0.772231,NaN,MIR1302-11
ENSG00000237613.2,4.917684,0.479224,1.333292,0.359430,0.719274,NaN,FAM138A
ENSG00000238009.2,155.704964,-0.305639,0.250297,-1.221106,0.222046,0.420962,RP11-34P13.7


Which genes have a significant value in the padj column? 

In [6]:
sig_genes = deseq2_result.loc[deseq2_result['padj'] < 0.05]

Let's separate those between upregulated and downregulated

In [7]:
sig_genes_up = sig_genes.loc[sig_genes['log2FoldChange'] > 1]
print sig_genes_up.shape
sig_genes_up.head()

(744, 7)


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,sym
ENSG00000230699.2,42.862475,1.858231,0.510016,3.643478,2.689790e-04,1.952094e-03,RP11-54O7.1
ENSG00000223764.2,8.957234,3.407760,1.225697,2.780263,5.431483e-03,2.431475e-02,RP11-54O7.3
ENSG00000187634.6,20.930982,4.708596,1.025346,4.592203,4.385918e-06,5.426083e-05,SAMD11
ENSG00000215915.5,11.335026,3.776213,1.157018,3.263746,1.099496e-03,6.520547e-03,ATAD3C
ENSG00000197530.8,233.131802,1.460990,0.239785,6.092917,1.108714e-09,3.394529e-08,MIB2


In [8]:
sig_genes_down = sig_genes.loc[sig_genes['log2FoldChange'] < -1]
print sig_genes_down.shape
sig_genes_down.head()

(226, 7)


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,sym
ENSG00000097021.15,310.091523,-1.372322,0.201248,-6.819059,9.163884e-12,4.135108e-10,ACOT7
ENSG00000131914.6,37.609710,-2.327575,0.532776,-4.368772,1.249469e-05,1.391319e-04,LIN28A
ENSG00000175130.6,2758.189342,-1.042473,0.098009,-10.636476,2.016108e-26,7.179096e-24,MARCKSL1
ENSG00000214114.4,462.320291,-1.043618,0.159934,-6.525288,6.787080e-11,2.628189e-09,MYCBP
ENSG00000157193.10,1011.776526,-1.116318,0.127472,-8.757379,1.998436e-18,2.922713e-16,LRP8


I want to save those geneIDs, now that I have called them as significant, I don't care about the rest of the stuff

In [9]:
upregulated_genes = sig_genes_up.index
downregulated_genes = sig_genes_down.index

We are going to use bedtools to intersect those genes with a list of peaks that we called from FTO clip. Check out the bedtools documentation. In particular, we are going to use bedtools intersect. 

In order to use bedtools intersect, we need a bed file of genes, not just a list of geneIDs. I put a bed file in the shared folder, let's load that in as a dataframe and make new bed files of genes that we are interested in

In [15]:
bedfile_of_genes = pd.read_table(peak_dir+"hg19_genes.bed",  
                              names = ['chrom','start','stop','geneid','name','strand'])
bedfile_of_genes.head()

,chrom,start,stop,geneid,name,strand
0,chr1,11869,14412,ENSG00000223972.4,DDX11L1,+
1,chr1,14363,29806,ENSG00000227232.4,WASH7P,-
2,chr1,29554,31109,ENSG00000243485.2,MIR1302-11,+
3,chr1,34554,36081,ENSG00000237613.2,FAM138A,-
4,chr1,52473,54936,ENSG00000268020.2,OR4G4P,+


I want to set the geneID as the index

In [16]:
bedfile_of_genes.set_index("geneid", drop=False, inplace=True)
bedfile_of_genes.head()

,chrom,start,stop,geneid,name,strand
geneid,,,,,,
ENSG00000223972.4,chr1,11869,14412,ENSG00000223972.4,DDX11L1,+
ENSG00000227232.4,chr1,14363,29806,ENSG00000227232.4,WASH7P,-
ENSG00000243485.2,chr1,29554,31109,ENSG00000243485.2,MIR1302-11,+
ENSG00000237613.2,chr1,34554,36081,ENSG00000237613.2,FAM138A,-
ENSG00000268020.2,chr1,52473,54936,ENSG00000268020.2,OR4G4P,+


How do I use this new index to grab only upregulated genes?

In [20]:
upregulated_bed = bedfile_of_genes.loc[upregulated_genes]

In [21]:
downregulated_bed = bedfile_of_genes.loc[downregulated_genes]

Let's save those files, but we don't want to save the index again or the header because bedfiles don't have a header. They also need to be tab separated

In [23]:
save_dir = "/home/ucsd-train01/projects/fto_shrna/fto_clip/"
upregulated_bed.to_csv(save_dir+"upregulated_genes.bed", index=None, header=None, sep="\t")
downregulated_bed.to_csv(save_dir+"downregulated_genes.bed", index=None, header=None, sep="\t")

One more thing, we need a bedfile of significant peaks to compare to these upregulated and downregulated genes. Let's load up the peak file, and filter for pvalue and fold change cutoffs

In [24]:
rep1_peaks = pd.read_table(peak_dir+"fto_clip_rep1.bed", index_col=0, 
                          names = ['chrom','start','stop','pval','fc','strand'])
rep1_peaks.head()

,start,stop,pval,fc,strand
chrom,,,,,
chr7,128502977,128503105,27.176232,2.874631,+
chr7,140396555,140396643,24.549890,3.739772,+
chr7,150066826,150066915,17.262688,3.449854,+
chr7,140402683,140402751,16.942373,4.468390,+
chr7,128388639,128388728,15.599877,2.498764,+


How do we select rows with pval greater than 3 and fold change greater than 2?

In [28]:
rep1_peaks_sig_peaks = rep1_peaks.loc[(rep1_peaks['pval'] > 3) &
               (rep1_peaks['fc'] > 2)]
rep1_peaks_sig_peaks.head()

,start,stop,pval,fc,strand
chrom,,,,,
chr7,128502977,128503105,27.176232,2.874631,+
chr7,140396555,140396643,24.549890,3.739772,+
chr7,150066826,150066915,17.262688,3.449854,+
chr7,140402683,140402751,16.942373,4.468390,+
chr7,128388639,128388728,15.599877,2.498764,+


In [30]:
rep1_peaks_sig_peaks.to_csv(save_dir+"fto_rep1_sig_peaks.bed", header=None, sep="\t")

Let's do the same thing for rep2 peaks

In [31]:
rep2_peaks = pd.read_table(peak_dir+"fto_clip_rep2.bed", index_col=0, 
                          names = ['chrom','start','stop','pval','fc','strand'])
rep2_peaks.head()

,start,stop,pval,fc,strand
chrom,,,,,
chr7,140396556,140396627,26.740645,3.866446,+
chr7,128502937,128503095,24.969441,2.631810,+
chr7,140396627,140396643,21.331381,4.526628,+
chr7,128388651,128388732,19.171018,2.849902,+
chr7,140402664,140402746,18.386657,4.667984,+


In [32]:
rep2_peaks_sig_peaks = rep2_peaks.loc[(rep2_peaks['pval'] > 3) &
               (rep2_peaks['fc'] > 2)]
rep2_peaks_sig_peaks.head()

,start,stop,pval,fc,strand
chrom,,,,,
chr7,140396556,140396627,26.740645,3.866446,+
chr7,128502937,128503095,24.969441,2.631810,+
chr7,140396627,140396643,21.331381,4.526628,+
chr7,128388651,128388732,19.171018,2.849902,+
chr7,140402664,140402746,18.386657,4.667984,+


In [33]:
rep2_peaks_sig_peaks.to_csv(save_dir+"fto_rep2_sig_peaks.bed", header=None, sep="\t")

Now we're ready to move onto bedtools.